## Import

In [1]:
import os
import yaml
import warnings
from typing import List

from utils import get_openai_api_key
from utils_data import TIMETABLE, add_slot, is_slot_conflict, remove_slot, access_timetable

from crewai import Agent, Task, Crew
from pydantic import BaseModel, Field
from crewai.tools import BaseTool


warnings.filterwarnings('ignore')

#### Get api key

In [2]:
openai_api_key = get_openai_api_key()

## Setup configs

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Pydantic models

In [4]:
class EventSlot(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")
    startTime: str = Field(..., description="Start time of the slot. The format is: HH:MM")
    endTime: str = Field(..., description="End time of the slot. The format is: HH:MM")
    title: str = Field(..., description="Short title to summarize the event's essential information.")

class MultipleSlots(BaseModel):
    slots: List[EventSlot] = Field(..., description="List of event slots")

## Tools

In [5]:
from datetime import date
from typing import Type


from datetime import datetime

class InputDate(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")

class ValidDateTool(BaseTool):
    name: str = "Valid Date Tool"
    description: str = "Tool that checks if the date and week day are valid"
    args_schema: Type[BaseModel] = InputDate

    def _run(self, date: str, weekday: str):
        try:
            datetime.strptime(date, "%d/%m/%Y")
        except ValueError:
            return f"Error: {str(e)}. Please provide a valid date."
        
        try:
            date_obj = datetime.strptime(date, "%d/%m/%Y")
            correct_weekday = date_obj.strftime("%A")  # Full name (e.g., "Monday")
            correct_weekday.lower().startswith(weekday.lower())
            return "Valid date"
        except ValueError as e:
            return f"Error: {str(e)}. The week day does not corresponds to the date"
        
class AddSlot(BaseTool):
    name: str = "Add Slot Tool"
    description: str = "Tool that adds slots to the timetable"
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        for eventSlot in slots:
            res = add_slot(eventSlot)
            if res != "SUCCESS":
                return res
        return "SUCCESS"
    
class RemoveSlot(BaseTool):
    name: str = "Remove Stol Tool"
    description: str = "Tool that removes slots to the timetable"
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        for eventSlot in slots:
            res = remove_slot(eventSlot)
            if res != "SUCCESS":
                return res
        return "SUCCESS"
    
class AccessTimetable(BaseTool):
    name: str = "Access Timetable"
    description: str = "Tool that enables access to the stringified timetable"
    
    def _run(self):
        return access_timetable()
            
    
class CheckConflict(BaseTool):
    name: str = "Check Conflict Tool"
    description: str = "Tool that checks if there is a conflict between an event and an already setup event in the timetable. " \
    "Returns the list of events where there is a conflict with already setup events in the timetable."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        conflicts = []
        for slot in slots:
            res = is_slot_conflict(slot)
            if res == "There is a conflict":
                conflicts.append(slot)
        return conflicts


## Agents

In [6]:
converter = Agent(
    config=agents_config["converter"],
    inject_date=True,
    date_format="%A, %B %d, %Y",
    tools=[ValidDateTool()])

assistant = Agent(
    config=agents_config["assistant"],
    tools=[AddSlot(), CheckConflict(), RemoveSlot(), AccessTimetable()]
)

## Tasks

In [7]:
conversion = Task(
    config=tasks_config["conversion"],
    agent=converter,
    output_pydantic=MultipleSlots
)

manage_timetable = Task(
    config=tasks_config["manage_timetable"],
    agent=assistant
)

## Crew

In [ ]:
crew = Crew(
    agents=[converter, assistant],
    tasks=[conversion, manage_timetable],
    verbose=True
)

## Execution

In [9]:
result = crew.kickoff(inputs={"event": "I have a meeting with my boss for the next two mondays at 5pm for an hour"})

In [10]:
TIMETABLE

{}

In [11]:
result

CrewOutput(raw='The slot for the meeting with my boss is already scheduled for the next two Mondays, October 6th, 2025, and October 13th, 2025, from 5 PM to 6 PM.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Gather the most crucial information about the event: I have a meeting with my boss for the next two mondays at 5pm for an hour. You should gather the following if there if:\n    1. Date in format DD/MM/YYYY\n    2. Start time in format HH:MM\n    3. End time in format HH:MM\n    4. Event title, generate if not explicit\nMake sure you are sending a valid date.\n\n\nCurrent Date: Friday, September 26, 2025', name='Gather the most crucial information about the event: I have a meeting with my boss for the next two mondays at 5pm for an hour. You should gather the following if there if:\n    1. Date in format DD/MM/YYYY\n    2. Start time in format HH:MM\n    3. End time in format HH:MM\n    4. Event title, generate if not explicit\nMake sure you are sending a 

In [12]:
# result.pydantic.model_dump()["slots"]

In [13]:
result.pydantic.model_dump()

AttributeError: 'NoneType' object has no attribute 'model_dump'

## Cost

In [ ]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0009


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,6131,5539,0,592,8
